In [1]:
import pandas as pd
import requests
import io
import numpy as np

from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import pickle
import warnings

warnings.filterwarnings("ignore")

# Downloading the csv file from your GitHub account


student = 'https://raw.githubusercontent.com/ongalajacob/Javic_ML/main/2021_data/javicjun_schms_table_student.csv'
class_session = 'https://raw.githubusercontent.com/ongalajacob/Javic_ML/main/2021_data/javicjun_schms_table_class_session.csv'
classregister = 'https://raw.githubusercontent.com/ongalajacob/Javic_ML/main/2021_data/javicjun_schms_table_classregister.csv'
invoice = 'https://raw.githubusercontent.com/ongalajacob/Javic_ML/main/2021_data/javicjun_schms_table_invoice.csv'
employees = 'https://raw.githubusercontent.com/ongalajacob/Javic_ML/main/2021_data/javicjun_schms_table_employees.csv'
fees = 'https://raw.githubusercontent.com/ongalajacob/Javic_ML/main/2021_data/javicjun_schms_table_fees.csv'
exams = 'https://raw.githubusercontent.com/ongalajacob/Javic_ML/main/2021_data/javicjun_schms_table_exams.csv'

stud_df = pd.read_csv(student)
session_df = pd.read_csv(class_session)
classregister_df = pd.read_csv(classregister)
invoice_df = pd.read_csv(invoice)
employees_df = pd.read_csv(employees)
fee_df = pd.read_csv(fees)
exams_df = pd.read_csv(exams)



In [29]:
Register = pd.merge(left=classregister_df, right=session_df, how='left', left_on='Session', right_on='id')
Register.drop(["id_y","Session"], axis=1, inplace=True)
Register.rename(columns = {'id_x':'id', }, inplace = True)

Register = pd.merge(left=Register, right=stud_df, how='left', left_on='student', right_on='ID')
Register.drop(['ID','Mother', 'Father', 'Guadian',
       'Class_Admitted', 'DOA', 'address',
        'DateExit',"student",'Startdate','Enddate','ClassTeacher'], axis=1, inplace=True)
Register.rename(columns = {'sex_x':'sex_teacher','sex_y':'sex_stud' ,'name':'name_stud' ,'Class_grd':'grade'  }, inplace = True)

Register=pd.merge(left=Register, right=invoice_df, how='left', left_on='id', right_on='ClassRegisterID')
Register.drop(["id_y"], axis=1, inplace=True)
Register.rename(columns = {'id_x':'id', }, inplace = True)
Register_df = Register.rename(columns=str.lower)
Register_df['tot_invoice']=Register_df['uniform']+Register_df['uniform_no']+(Register_df['transport']*Register_df['transport_months'])+ \
     (Register_df['lunch']*Register_df['lunch_months'])+Register_df['otherlevyindv']+Register_df['tutionfee']+Register_df['examfee']+ \
         Register_df['booklevy'] +Register_df['activityfee']+Register_df['otherlevies']
Register_df= Register_df[['id', 'year', 'term', 'grade', 
       'name_stud', 'adm', 'dob', 'sex', 'phone1', 'phone2', 'phone3', 'enrolstatus', 'bal_bf', 'tot_invoice']] 
Register_df["grade"].replace({"Baby1":'Baby','Class5':'Grade5',}, inplace=True)
Register_df['grade'] = pd.Categorical(Register_df['grade'], ['Baby', 'PP1', 'PP2','Grade1',  'Grade2', 'Grade3', 'Grade4', 'Grade5'])     


fee_df[['Admission', 'Tuition',
       'Transport', 'Uniform', 'Lunch', 'Exams', 'BookLvy', 'Activity',
       'OtheLvy']] = fee_df[['Admission', 'Tuition',
       'Transport', 'Uniform', 'Lunch', 'Exams', 'BookLvy', 'Activity',
       'OtheLvy']].fillna(0)
fee_df["total_paid"] =fee_df["Admission"] +fee_df["Tuition"] +fee_df["Transport"] +fee_df["Uniform"] \
       +fee_df["Lunch"] +fee_df["Exams"] +fee_df["BookLvy"] +fee_df["Activity"] +fee_df["OtheLvy"] 
fee_df['id']=fee_df['id'].astype(object)
fee_df['ReceiptNo']=fee_df['ReceiptNo'].astype(object)
#fee_df['DOP1']=pd.to_datetime(fee_df['DOP'] ,format = '%Y-%m-%d') 
fee_df['DOP']=pd.to_datetime(fee_df['DOP']).dt.strftime('%Y-%m-%d')
fees_df=pd.merge(left=fee_df, right=Register_df, how='left', left_on='ClassRegisterID', right_on='id')
fees_df = fees_df.rename(columns=str.lower)
fees_df.drop(["id_y",'classregisterid'], axis=1, inplace=True)
fees_df.rename(columns = {'id_x':'id', }, inplace = True)
fees_df=fees_df[['id', 'receiptno', 'dop', 'year', 'term', 'grade','adm','name_stud', 'enrolstatus', 'phone1', 'phone2', 'phone3', 'admission', 'tuition', 'transport',
      'uniform', 'lunch', 'exams', 'booklvy', 'activity', 'othelvy',  'total_paid']]


fee_df1=fee_df.groupby(["ClassRegisterID"])["total_paid"].sum().reset_index(name='Total_collection')
fees_bal_df=pd.merge(left=fee_df1, right=Register_df, how='left', left_on='ClassRegisterID', right_on='id')
fees_bal_df["bal_cf"] =fees_bal_df["tot_invoice"] - fees_bal_df["Total_collection"] 
fees_bal_df=fees_bal_df[['id','year', 'term','grade',  'adm','name_stud','enrolstatus',  'bal_bf', 'tot_invoice', 'Total_collection','bal_cf' , 'phone1', 'phone2', 'phone3']]
fees_bal_df = fees_bal_df.rename(columns=str.lower)


exam_df=pd.merge(left=exams_df, right=Register_df, how='left', left_on='ClassRegisterID', right_on='id')
exam_df.drop(["id_y",'ClassRegisterID'], axis=1, inplace=True)
exam_df.rename(columns = {'id_x':'id', }, inplace = True)
exam_df=exam_df[['id', 'ExamType', 'year', 'term', 'grade','adm',   'name_stud', 'dob', 'sex','enrolstatus',  'Maths', 'EngLan', 'EngComp',
       'KisLug', 'KisIns', 'Social', 'Creative', 'CRE', 'Science', 'HmScie',
       'Agric', 'Music', 'PE']]
exam_df= exam_df.rename(columns=str.lower)
subjects=['maths', 'englan', 'engcomp', 'kislug', 'kisins', 'social', 'creative',
       'cre', 'science', 'hmscie', 'agric', 'music', 'pe']
exam_df['Tot_marks'] = exam_df[subjects].sum(numeric_only=True, axis=1)
exam_df=exam_df.dropna()
exam_df['year'] = exam_df.year.astype(int) 
exam_df['adm'] = exam_df.adm.astype(int)  

In [31]:
fees_bal_df.columns

Index(['id', 'year', 'term', 'grade', 'adm', 'name_stud', 'enrolstatus',
       'bal_bf', 'tot_invoice', 'total_collection', 'bal_cf', 'phone1',
       'phone2', 'phone3'],
      dtype='object')

In [20]:
fees_df[fees_df.enrolstatus=='In_Session'][['receiptno', 'dop', 'grade', 'adm', 'name_stud',
            'admission', 'tuition', 'transport', 'uniform', 'lunch',
            'exams', 'booklvy', 'activity', 'othelvy', 'total_paid']]  

,receiptno,dop,grade,adm,name_stud,admission,tuition,transport,uniform,lunch,exams,booklvy,activity,othelvy,total_paid
0,6598,2020-10-19,Grade4,54,Reinhard Levi Onyango,0.0,1900.0,900.0,0.0,0.0,0.0,200.0,0.0,0.0,3000.0
1,6597,2020-10-14,Grade4,252,Meths Rino Nyagol,0.0,2000.0,0.0,0.0,600.0,100.0,200.0,100.0,0.0,3000.0
3,6596,2020-10-14,Grade4,226,Katheryn Fejenia Wanyama,0.0,3000.0,3300.0,700.0,1800.0,100.0,200.0,100.0,0.0,9200.0
4,6600,2020-10-19,Grade4,113,Shantel Whitney Otieno,0.0,1650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1650.0
5,6602,2020-10-19,Grade4,282,Kerry Dalmas Otieno,0.0,1700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,7820,2021-05-26,Grade1,211,Dalmus Ojwang Odhiambo,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0
1165,7821,2021-05-27,Grade3,287,Gabriel Ebeneza,0.0,3700.0,3300.0,0.0,1800.0,200.0,200.0,100.0,0.0,9300.0
1166,7822,2021-05-28,Grade2,255,Byron John Onyango,0.0,400.0,0.0,0.0,600.0,200.0,200.0,100.0,0.0,1500.0
1167,7823,2021-05-28,PP1,285,Sharlet Kimberly Odede,0.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800.0


In [ ]:
(['ADM', 'name',  'Mother', 'Father', 'Guadian', 'DOB', 'sex',
        'PHONE1', 'PHONE2', 'PHONE3', 'DOA']